In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import gc
import math
from gensim import corpora, models, utils
from gensim.models import LdaModel, CoherenceModel
import jieba

C:\Users\Archur\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Archur\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Archur\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Archur\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: P

In [2]:
df = pd.read_csv("10802_2020 NewTaipeiCity Legislators candidates.csv") #匯資料
df["Post_Date"] = df["Date"].str.split(" ").str.get(0).str.title() #加Post_Date欄位
df['Post_Date'] = pd.to_datetime(df['Post_Date'])

In [3]:
df

,Date,Page_Name,Page_ID,Link,Type,All_Reaction_Count,LIKE_COUNT,WOW_COUNT,LOVE_COUNT,HAHA_COUNT,SAD_COUNT,ANGRY_COUNT,Comment_Count,Share_Count,Message,Link_Title,Link Description,created_time,Post_Date
0,2019/11/01 09:44:03,余天 Yu Tian,184889114896212,NaN,video,141559,131345,71,8086,34,2003,20,14917,1975,「女兒妳堅強，我就放心」\n「爸爸，你也要顧好自己的身體」\n\n擔心女兒化療吃不下\n這幾...,NaN,NaN,1572572643000,2019-11-01
1,2019/09/18 17:38:31,賴品妤,102499747817312,NaN,photo,27126,23611,46,1647,1766,5,51,1916,1444,很多人是從五年前的三一八運動才開始認識賴品妤，那天晚上我用雙手撐住立法院議場的大門，阻擋服貿...,NaN,NaN,1568799511000,2019-09-18
2,2020/01/10 17:21:05,余天 Yu Tian,184889114896212,NaN,video,22177,21529,11,456,22,120,39,785,278,選前之夜彩蛋搶先曝光！\n\n余天給三重的一封信！\n余苑綺：爸爸加油！我一定會好起來，謝謝...,余天給三重的一封信！,NaN,1578648065000,2020-01-10
3,2020/01/07 22:41:02,羅致政粉絲團,182836328407031,NaN,video,17915,17353,16,347,122,10,67,1439,1034,鍾年晃是資深媒體人，也是我的苗栗同鄉。\n他來幫我助選時，說了一段發人深省話，他說：民進黨每...,鍾年晃：下架吳斯懷還不夠，我們要下架國民黨！,NaN,1578408062000,2020-01-07
4,2020/01/11 11:51:36,賴品妤,102499747817312,NaN,photo,19769,17059,13,1790,900,2,5,857,86,剛剛我已經把票投好囉😍\n怎麼還有人沒出門勒😗,NaN,NaN,1578714696000,2020-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12603,2019/12/03 20:47:57,黃志雄&洪佳君,192038527523436,NaN,status,0,0,0,0,0,0,0,0,0,#用熱情的力量翻轉地方\n\n有人形容，熱情就是太陽。因為在這個環境裡，熱情會延伸許多好的...,NaN,NaN,1575377277000,2019-12-03
12604,2019/12/07 17:59:37,羅明才,186040511466669,NaN,status,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,1575712777000,2019-12-07
12605,2019/12/18 10:52:38,余天 Yu Tian,184889114896212,NaN,photo,0,0,0,0,0,0,0,0,0,NaN,余天 Yu Tian's cover photo,NaN,1576637558000,2019-12-18
12606,2019/12/18 21:47:32,柯志恩,911433142308383,NaN,photo,0,0,0,0,0,0,0,0,0,NaN,柯志恩's cover photo,NaN,1576676852000,2019-12-18


In [4]:
df["Page_Name"] = df["Page_Name"].str.replace("賴品妤","太陽花女神")

In [5]:
df

,Date,Page_Name,Page_ID,Link,Type,All_Reaction_Count,LIKE_COUNT,WOW_COUNT,LOVE_COUNT,HAHA_COUNT,SAD_COUNT,ANGRY_COUNT,Comment_Count,Share_Count,Message,Link_Title,Link Description,created_time,Post_Date
0,2019/11/01 09:44:03,余天 Yu Tian,184889114896212,NaN,video,141559,131345,71,8086,34,2003,20,14917,1975,「女兒妳堅強，我就放心」\n「爸爸，你也要顧好自己的身體」\n\n擔心女兒化療吃不下\n這幾...,NaN,NaN,1572572643000,2019-11-01
1,2019/09/18 17:38:31,太陽花女神,102499747817312,NaN,photo,27126,23611,46,1647,1766,5,51,1916,1444,很多人是從五年前的三一八運動才開始認識賴品妤，那天晚上我用雙手撐住立法院議場的大門，阻擋服貿...,NaN,NaN,1568799511000,2019-09-18
2,2020/01/10 17:21:05,余天 Yu Tian,184889114896212,NaN,video,22177,21529,11,456,22,120,39,785,278,選前之夜彩蛋搶先曝光！\n\n余天給三重的一封信！\n余苑綺：爸爸加油！我一定會好起來，謝謝...,余天給三重的一封信！,NaN,1578648065000,2020-01-10
3,2020/01/07 22:41:02,羅致政粉絲團,182836328407031,NaN,video,17915,17353,16,347,122,10,67,1439,1034,鍾年晃是資深媒體人，也是我的苗栗同鄉。\n他來幫我助選時，說了一段發人深省話，他說：民進黨每...,鍾年晃：下架吳斯懷還不夠，我們要下架國民黨！,NaN,1578408062000,2020-01-07
4,2020/01/11 11:51:36,太陽花女神,102499747817312,NaN,photo,19769,17059,13,1790,900,2,5,857,86,剛剛我已經把票投好囉😍\n怎麼還有人沒出門勒😗,NaN,NaN,1578714696000,2020-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12603,2019/12/03 20:47:57,黃志雄&洪佳君,192038527523436,NaN,status,0,0,0,0,0,0,0,0,0,#用熱情的力量翻轉地方\n\n有人形容，熱情就是太陽。因為在這個環境裡，熱情會延伸許多好的...,NaN,NaN,1575377277000,2019-12-03
12604,2019/12/07 17:59:37,羅明才,186040511466669,NaN,status,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,1575712777000,2019-12-07
12605,2019/12/18 10:52:38,余天 Yu Tian,184889114896212,NaN,photo,0,0,0,0,0,0,0,0,0,NaN,余天 Yu Tian's cover photo,NaN,1576637558000,2019-12-18
12606,2019/12/18 21:47:32,柯志恩,911433142308383,NaN,photo,0,0,0,0,0,0,0,0,0,NaN,柯志恩's cover photo,NaN,1576676852000,2019-12-18


In [6]:
df_green = df[df['Page_Name']=='太陽花女神']
df_green_New = df_green[['Post_Date', 'Message']]
df_green_New = df_green_New.dropna()
df_green_New['Message']

doc_names = range(len(df_green_New))
text_list = list(df_green_New['Message'])
df_green_docs = pd.DataFrame(columns=['jieba_results'])
df_green_docs['jieba_results'] = df_green_docs['jieba_results'].astype('str')

In [11]:
#停用字和自訂字典
jieba.load_userdict("user.txt")
stopwords = [line.strip() for line in open('stopword.txt', 'r', encoding='utf-8').readlines()]  

for i in doc_names:
    
    words = jieba.cut(text_list[i], cut_all = False)
    text = ''
    for word in words:
        if word not in stopwords: 
            text = text + ' ' + str(word)
    s = pd.Series({'jieba_results': text})
    df_green_docs = df_green_docs.append(s, ignore_index=True)  
    
df_green_New.reset_index(inplace=True)  
df_green_New['jieba_results'] = df_green_docs
df_green_New['df_class'] = 'green'

In [12]:
green_file = df_green_New['jieba_results']
green_file

0       五年 前 三一八 運動 認識 賴品妤 晚上 我用 雙手 撐 住 立法院 議場 大門 阻擋 ...
1                          剛剛 票 投好 囉 😍 \n 怎麼還 有人 沒出 門勒 😗
2       這幾天 收到 一份 最新 民調 想 久 覺得 報告 一下 選情 \n 　 \n 這份民調 ...
3       🐭 🐭 爆竹 一聲 舊 歲 品妤 先 祝 🐭 🐭 \n   鼠 勾以 捏   鼠給   大...
4       距離 依法 停止 競選 活動 時間 倒數 分享 這段 造勢 晚會 說 話 \n \n  ...
                             ...                        
196     🎮 擔任 遊戲 技術 指導 \n ＃ 總統府 大冒險     🎯 線上 直播 😁 \n 快...
197     謝謝 鞠躬 \n 　 \n 返校 包場 報名 太 踴躍 開表單 驚 呆 😮 \n 　 \n...
198      LIVE 預告 ｜ 明早 10 : 00 \n \n ＃ 敢衝 敢 拼     ＃ 敬請 期待
199     ＃ 護國 保台   ＃ 野百合 太陽 花都 \n 　 \n 跨 世代 學運 大串 💪 \n...
200     超級 感謝 支持者   張博凱   熱心 🔥 幫 做 這支 影片 \n 太感動 😭 預告 ...
Name: jieba_results, Length: 201, dtype: object

In [13]:
green_list = []
for file in green_file:
            #simple_preprocess: tokenize,去除標點 - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15) 
            green_list.append(list(utils.simple_preprocess(file, deacc=True, min_len=2,)))
            #print(file)
#print(green_list[:1])

In [14]:
from gensim import corpora, models, utils
from gensim.models import LdaModel, CoherenceModel

In [15]:
dict = corpora.Dictionary(green_list)
print("dict = ", dict)
print("------------------------------")

dict =  Dictionary(4914 unique tokens: ['一個標', '一戰', '一票', '一票票', '一途']...)
------------------------------


In [16]:
# creat mapping matrix of [word_id, word_frequency] for each word segment
corpus = [dict.doc2bow(text) for text in green_list]   

#上面這邊印出每個字的詞頻
#(a,b) a是編號第幾個字 / b是那個字出現了幾次
print("corpus[:1] = ", corpus[:1])   
print("------------------------------")

# 下面是哪個字具體出現了幾次
print("print lists of [word:frequency]")
[[print((dict[id], freq)) for id,freq in cp] for cp in corpus[:1]]
print("================end of the ouput================")

corpus[:1] =  [[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 2), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 5), (31, 2), (32, 1), (33, 1), (34, 2), (35, 3), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 3), (53, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 2), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 3), (76, 1), (77, 1), (78, 5), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 3), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 2), (105, 1), (106, 1), (107, 1), (108, 1), (109

In [23]:
# 設定模型分類的參數
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word= dict, num_topics=10, passes=20, alpha='auto', chunksize=50, per_word_topics=True)
output = open("HW_LDA", 'w', encoding='utf-8-sig')

#把分好的主題關鍵字印出來 #印10個主題，一個主題20個字
lstTM = lda_model.print_topics(num_topics=10, num_words=20)
for tupleTM in lstTM:
        str1 = "".join(str(x) for x in tupleTM)
        #print("type(str1) = ", type(str1))
        str1 += "\n"
        print("strTM = ", str1) #把主題內的代表字印出來
        output.write(str1)
        
output.close()
 
print("------------------------------")
print("Perplexity = ", lda_model.log_perplexity(corpus))

print("------------------------------")
cm = CoherenceModel(model=lda_model, texts=list(green_file), dictionary=dict, coherence= 'c_v' )
coher_lda = cm.get_coherence()
print("\n Coherence Score: ", coher_lda) #印出聚合分數，越高越好

strTM =  00.041*"下午" + 0.013*"交通" + 0.012*"問題" + 0.009*"活動" + 0.008*"未來" + 0.008*"方式" + 0.008*"特色" + 0.008*"重要" + 0.007*"觀光" + 0.007*"汐止" + 0.007*"規劃" + 0.007*"發展" + 0.006*"道路" + 0.006*"設施" + 0.006*"主張" + 0.006*"台北市" + 0.006*"解決" + 0.005*"願景" + 0.005*"勝利" + 0.005*"成為"

strTM =  10.021*"瑞芳" + 0.016*"鄉親" + 0.012*"議員" + 0.011*"地方" + 0.010*"新北市" + 0.009*"台灣" + 0.009*"萬里" + 0.008*"金山" + 0.008*"平溪" + 0.008*"年輕人" + 0.008*"政策" + 0.008*"主委" + 0.007*"國會" + 0.007*"貢寮" + 0.007*"年輕" + 0.006*"雙溪" + 0.006*"未來" + 0.006*"汐止" + 0.006*"投品妤" + 0.006*"其實"

strTM =  20.023*"台灣" + 0.018*"香港" + 0.011*"支持" + 0.008*"投票" + 0.006*"民進黨" + 0.006*"未來" + 0.006*"社會" + 0.006*"民主" + 0.006*"政治" + 0.005*"先生" + 0.005*"守護" + 0.005*"生活" + 0.005*"民主自由" + 0.005*"一票" + 0.005*"賴清德" + 0.004*"候選人" + 0.004*"世代" + 0.004*"台灣人" + 0.004*"可能" + 0.004*"自由"

strTM =  30.010*"溫暖" + 0.008*"習慣" + 0.008*"記得" + 0.007*"投給" + 0.006*"國會" + 0.005*"立委" + 0.005*"責任" + 0.005*"恭喜" + 0.005*"加油" + 0.004*"這塊" + 0.004*"激動" + 0.004*"好多" + 0.004*"遇到" + 0.00

C:\Users\Archur\anaconda3\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:202: RuntimeWarning: invalid value encountered in true_divide
  numerator = (co_occur_count / num_docs) + EPSILON
C:\Users\Archur\anaconda3\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:203: RuntimeWarning: invalid value encountered in true_divide
  denominator = (w_prime_count / num_docs) * (w_star_count / num_docs)
C:\Users\Archur\anaconda3\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:198: RuntimeWarning: invalid value encountered in true_divide
  co_doc_prob = co_occur_count / num_docs



 Coherence Score:  nan
